In [1]:
import matplotlib.pyplot as plt
import geopandas as gp
import libpysal as ps
# from pysal.contrib.viz import mapping as maps
from mapclassify import Quantiles, FisherJenks, FisherJenksSampled
import pandas as pd
from cartopy import crs as ccrs
import folium
from folium import plugins
from datetime import datetime
import contextily as cx

/home/az/.cache/pypoetry/virtualenvs/volitve-6ym7LhX2-py3.8/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


Prebereimo shapefile z volilnimi enotami. Struktura fajla https://www.e-prostor.gov.si/fileadmin/struktura/RPE_struktura.docx

Stolpci:
* ENOTA: Šifra enote ( VE = državnozborska volilna enota, VO = državnozborski volilni okraj, VD = državnozborsko volišče)
* VDV_MID: Identifikator enote za državnozborske volitve
* VDV_ID: Šifra enote za državnozborske volitve
* VDV_UIME: Ime enote za državnozborske volitve uradno
* VDV_DJ: Ime enote za državnozborske volitve dvojezično
* POV_KM2: Površina enote za državnozborske volitve v km2
* D_OD: Datum veljavnosti
* DV_OD: Datum vnosa v bazo
* N8: Osemmestna šifra (sestavljena iz šifre državnozborske volilne enote – 3 mesta, šifre volilnega okraja – 2 mesti in šifre državnozborskega volišča – 3 mesta)
* C1: Prazno polje
* STATUS: Status veljavnosti zapisa (V – veljavno stanje)

Projekcije:
* EPSG:4326 = WGS84 (kar je ponavadi na netu)
* EPSG:3794 = D96 (nas lokalni sistem)



In [2]:
df_vdv = gp.read_file("data/VDV.geojson")
df_vdv = df_vdv[df_vdv.STATUS == 'V'] #filtriramo veljavna
df_vdv.to_crs('EPSG:4326', inplace=True)
df_vdv["centroid"] = gp.points_from_xy(df_vdv.CEN_E, df_vdv.CEN_N, crs='EPSG:3794').to_crs('EPSG:4326')
df_vdv["VE"] = pd.to_numeric(df_vdv["VDV_ID"] / 1000, downcast='integer').astype(int)
df_vdv["VO"] = pd.to_numeric(df_vdv["VDV_ID"] % 1000, downcast='integer').astype(int)
df_vdv.drop(['VDV_MID', 'C1', 'CEN_E', 'CEN_N', 'D_OD', 'DV_OD', 'VDV_DJ'], axis=1, inplace=True)
df_vdv.head()

,ENOTA,VDV_ID,VDV_UIME,POV_KM2,N8,STATUS,geometry,centroid,VE,VO
0,VE,1000,Volilna enota Kranj,2851.55,None,V,"POLYGON ((13.90794 46.51983, 13.90772 46.51973...",POINT (14.06017 46.43657),1,0
1,VE,2000,Volilna enota Postojna,3916.77,None,V,"POLYGON ((13.64633 45.53324, 13.64620 45.53317...",POINT (13.73514 46.18574),2,0
2,VE,3000,Volilna enota Ljubljana Center,1530.03,None,V,"POLYGON ((14.50134 45.60783, 14.50187 45.60807...",POINT (14.50504 46.05237),3,0
3,VE,4000,Volilna enota Ljubljana Bežigrad,1882.16,None,V,"POLYGON ((14.59867 45.62945, 14.59867 45.62945...",POINT (14.60168 46.14587),4,0
4,VE,5000,Volilna enota Celje,2551.39,None,V,"POLYGON ((15.04395 46.19751, 15.04409 46.19747...",POINT (14.96423 46.54370),5,0


In [3]:
df_ve = df_vdv[df_vdv.ENOTA == 'VE'] # filtriramo samo volilne enote
df_ve.set_index('VE', inplace=True)
df_ve.drop(['N8'], axis=1, inplace=True)
df_ve.head(n=10)

/home/az/.cache/pypoetry/virtualenvs/volitve-6ym7LhX2-py3.8/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,ENOTA,VDV_ID,VDV_UIME,POV_KM2,STATUS,geometry,centroid,VO
VE,,,,,,,,
1,VE,1000,Volilna enota Kranj,2851.55,V,"POLYGON ((13.90794 46.51983, 13.90772 46.51973...",POINT (14.06017 46.43657),0
2,VE,2000,Volilna enota Postojna,3916.77,V,"POLYGON ((13.64633 45.53324, 13.64620 45.53317...",POINT (13.73514 46.18574),0
3,VE,3000,Volilna enota Ljubljana Center,1530.03,V,"POLYGON ((14.50134 45.60783, 14.50187 45.60807...",POINT (14.50504 46.05237),0
4,VE,4000,Volilna enota Ljubljana Bežigrad,1882.16,V,"POLYGON ((14.59867 45.62945, 14.59867 45.62945...",POINT (14.60168 46.14587),0
5,VE,5000,Volilna enota Celje,2551.39,V,"POLYGON ((15.04395 46.19751, 15.04409 46.19747...",POINT (14.96423 46.54370),0
6,VE,6000,Volilna enota Novo mesto,3409.86,V,"POLYGON ((14.86480 45.98936, 14.86488 45.98919...",POINT (15.04906 46.14970),0
7,VE,7000,Volilna enota Maribor,1540.49,V,"POLYGON ((15.46912 46.61284, 15.46861 46.61320...",POINT (15.64398 46.56247),0
8,VE,8000,Volilna enota Ptuj,2589.04,V,"POLYGON ((16.27445 46.87230, 16.27394 46.87230...",POINT (15.99260 46.67522),0


In [4]:
df_vo = df_vdv[df_vdv.ENOTA == 'VO'] # filtriramo samo volilne okraje
df_vo.set_index(['VE', 'VO'], inplace=True)
df_vo.drop(['N8'], axis=1, inplace=True)
df_vo.head(n=15)

/home/az/.cache/pypoetry/virtualenvs/volitve-6ym7LhX2-py3.8/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


ENOTA  VDV_ID                     VDV_UIME  POV_KM2 STATUS  \
VE VO                                                              
1  1     VO    1001       Volilni okraj Jesenice   374.88      V   
   2     VO    1002   Volilni okraj Radovljica 1   509.77      V   
   3     VO    1003   Volilni okraj Radovljica 2   131.20      V   
   4     VO    1004        Volilni okraj Kranj 1    29.64      V   
   5     VO    1005        Volilni okraj Kranj 2    86.72      V   
   6     VO    1006        Volilni okraj Kranj 3   336.80      V   
   7     VO    1007          Volilni okraj Tržič   155.36      V   
   8     VO    1008  Volilni okraj Škofja Loka 1   146.03      V   
   9     VO    1009  Volilni okraj Škofja Loka 2   366.33      V   
   10    VO    1010         Volilni okraj Kamnik   289.68      V   
   11    VO    1011         Volilni okraj Idrija   425.14      V   
2  1     VO    2001         Volilni okraj Tolmin   942.29      V   
   2     VO    2002          Volilni okraj Piran    43.50      V   
   3     VO    2003          Volilni okraj Izola    28.50      V   
   4     VO    2004        Volilni okraj Koper 1    13.99      V   

                                                geometry  \
VE VO                                                      
1  1   POLYGON ((13.90794 46.51983, 13.90772 46.51973...   
   2   POLYGON ((13.81248 46.23836, 13.81254 46.23836...   
   3   POLYGON ((14.26226 46.32228, 14.26258 46.32242...   
   4   POLYGON ((14.35005 46.29351, 14.35027 46.29318...   
   5   POLYGON ((14.39601 46.20523, 14.39546 46.20580...   
   6   POLYGON ((14.50267 46.42040, 14.50249 46.42035...   
   7   POLYGON ((14.34969 46.43722, 14.34882 46.43722...   
   8   POLYGON ((14.31592 46.19769, 14.31567 46.19775...   
   9   POLYGON ((14.12148 46.00607, 14.12144 46.00590...   
   10  POLYGON ((14.53567 46.35757, 14.53560 46.35727...   
   11  POLYGON ((13.90963 45.97040, 13.90980 45.96949...   
2  1   POLYGON ((13.65242 46.44447, 13.65218 46.44463...   
   2   POLYGON ((13.56569 45.53047, 13.56552 45.53048...   
   3   POLYGON ((13.64633 45.53324, 13.64620 45.53317...   
   4   POLYGON ((13.74130 45.55770, 13.73970 45.55764...   

                        centroid  
VE VO                             
1  1   POINT (14.06017 46.43657)  
   2   POINT (13.98173 46.42060)  
   3   POINT (14.17207 46.34252)  
   4   POINT (14.35575 46.24364)  
   5   POINT (14.24194 46.26868)  
   6   POINT (14.46869 46.38189)  
   7   POINT (14.30871 46.36573)  
   8   POINT (14.30303 46.16721)  
   9   POINT (14.17628 46.25248)  
   10  POINT (14.61083 46.22608)  
   11  POINT (14.02210 46.00131)  
2  1   POINT (13.73514 46.18574)  
   2   POINT (13.57019 45.52865)  
   3   POINT (13.66177 45.53786)  
   4   POINT (13.72943 45.54655)

In [5]:
df_dv = df_vdv[df_vdv.ENOTA == 'VD'] # filtriramo samo volisca
# df_dv = df_dv.drop(['ENOTA', 'VDV_DJ', 'D_OD', 'DV_OD', 'C1', 'CEN_N', 'CEN_E', 'STATUS'], axis=1)
df_dv['N8'] = df_dv['N8'].astype(int)
df_dv.head()

/home/az/.cache/pypoetry/virtualenvs/volitve-6ym7LhX2-py3.8/lib/python3.8/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,ENOTA,VDV_ID,VDV_UIME,POV_KM2,N8,STATUS,geometry,centroid,VE,VO
96,VD,26704,"OŠ Žitečka vas, Zg. Duplek 98",6.55,707001,V,"POLYGON ((15.74382 46.51639, 15.74386 46.51651...",POINT (15.73274 46.50922),26,704
97,VD,26705,"Občina Duplek, Trg slovenske osamosvojitve 1",4.84,707002,V,"POLYGON ((15.74382 46.51639, 15.74357 46.51634...",POINT (15.74674 46.50527),26,705
98,VD,26706,"Osnovna šola Dvorjane, Dvorjane 15",5.40,707003,V,"POLYGON ((15.78827 46.50981, 15.78749 46.50983...",POINT (15.77168 46.49168),26,706
99,VD,26707,"Kulturni dom Vurberk, Vurberk 94",4.57,707004,V,"POLYGON ((15.79558 46.50124, 15.79552 46.50106...",POINT (15.78843 46.48644),26,707
100,VD,26718,Večnamenski prostor pri igrišču Zimica 46c,4.05,707005,V,"POLYGON ((15.75707 46.53788, 15.75701 46.53812...",POINT (15.75500 46.53543),26,718


Preberemo rezultate volitev v DZ 2018. 

In [6]:
dz18 = pd.read_csv('data/dz-2018.csv', skipfooter=1, dtype={"VOLIVCEV": int, "GImenik": int, "GLASOVALO": int, "Oddanih": int, "Imenik": int, "VELJAVNIH": int}, engine='python')
dz18["N8"] = pd.Series(dz18["VE"] * 100000 + dz18["VO"] * 1000 + dz18["Stv"]).astype(int)
dz18['VDV_ID'] = 1000 * dz18['VE'] + dz18['VO']
dz18.tail()

,VE,ENOTA,VO,OKRAJ,Stv,Volisce,Imenik,Potrdilo,VOLIVCEV,GImenik,...,SMC,SPS,SD,SOLIDARNOST,SSN,STRANKA AB,ZD,ZDRUŽENA LEVICA,N8,VDV_ID
6440,8,PTUJ,7,LENART,996,DKP volišča,0,0,0,8,...,1,0,1,0,0,0,0,2,807996,8007
6441,8,PTUJ,8,PESNICA,996,DKP volišča,0,0,0,6,...,1,0,0,0,1,0,0,0,808996,8008
6442,8,PTUJ,9,PTUJ,996,DKP volišča,0,0,0,5,...,0,0,2,0,0,0,0,0,809996,8009
6443,8,PTUJ,10,PTUJ,996,DKP volišča,0,0,0,13,...,1,0,0,0,0,1,1,1,810996,8010
6444,8,PTUJ,11,PTUJ,996,DKP volišča,0,0,0,4,...,1,0,2,0,0,0,0,0,811996,8011


In [7]:
dz18.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6445 entries, 0 to 6444
Data columns (total 42 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   VE                6445 non-null   int64 
 1   ENOTA             6445 non-null   object
 2   VO                6445 non-null   int64 
 3   OKRAJ             6445 non-null   object
 4   Stv               6445 non-null   int64 
 5   Volisce           6445 non-null   object
 6   Imenik            6445 non-null   int64 
 7   Potrdilo          6445 non-null   int64 
 8   VOLIVCEV          6445 non-null   int64 
 9   GImenik           6445 non-null   int64 
 10  GPotrdilo         6445 non-null   int64 
 11  GLASOVALO         6445 non-null   int64 
 12  Oddanih           6445 non-null   int64 
 13  Neveljavnih       6445 non-null   int64 
 14  VELJAVNIH         6445 non-null   int64 
 15  ZELENI            6445 non-null   int64 
 16  DeSUS             6445 non-null   int64 
 17  DD            

In [8]:
candidates = set(dz18.columns.values) - set(['VE', 'ENOTA', 'VO', 'OKRAJ', 'Stv', 'Volisce', 'Imenik', 'Potrdilo',
       'VOLIVCEV', 'GImenik', 'GPotrdilo', 'GLASOVALO', 'Oddanih',
       'Neveljavnih', 'VELJAVNIH', 'N8', 'VDV_ID'])
print(candidates)

{'Zsi', 'LNBP', 'SSN', 'ReSET', 'LEVICA', 'NSi', 'STRANKA AB', 'SNS', 'LMŠ', 'DD', 'ZELENI', 'SLS', 'SOLIDARNOST', 'DeSUS', 'ZDRUŽENA DESNICA', 'GSN', 'NPS', 'SPS', 'PIRATI', 'SDS', 'ZDRUŽENA LEVICA', 'SD', 'ZD', 'SMC', 'GAS'}


In [9]:
dz18_2 = dz18[candidates.union(set(['VDV_ID', 'GLASOVALO', 'Oddanih', 'VELJAVNIH', 'Neveljavnih']))]
dz18_2.sum(axis=0) / dz18_2.sum(axis=0)['VELJAVNIH'] * 100

Zsi                    0.593314
LNBP                   0.879253
VELJAVNIH            100.000000
SSN                    0.138818
Oddanih              101.162275
ReSET                  0.412076
LEVICA                 9.326482
NSi                    7.158817
STRANKA AB             5.105168
Neveljavnih            1.162275
ZDRUŽENA DESNICA       0.240266
SNS                    4.172610
VDV_ID              3320.694492
LMŠ                   12.596833
GSN                    0.487601
DD                     1.519475
NPS                    0.020985
SPS                    0.174055
PIRATI                 2.152628
SDS                   24.917826
ZDRUŽENA LEVICA        0.569186
SD                     9.934272
ZD                     0.622603
ZELENI                 1.089444
SMC                    9.748434
GLASOVALO            101.168784
GAS                    0.351477
SLS                    2.618009
SOLIDARNOST            0.245091
DeSUS                  4.925277
dtype: float64

In [10]:
dz18_vo = dz18.groupby(['VDV_ID']).sum()

# # dz18_vo['PIRATI_DELEZ'] = dz18_vo['PIRATI'] / dz18_vo['VELJAVNIH'] * 100
dz18_vo.reset_index(inplace=True)
for candidate in candidates:
    dz18_vo['{}_DELEZ'.format(candidate)] = dz18_vo[candidate] / dz18_vo['VELJAVNIH'] * 100
dz18_vo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88 entries, 0 to 87
Data columns (total 64 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   VDV_ID                  88 non-null     int64  
 1   VE                      88 non-null     int64  
 2   VO                      88 non-null     int64  
 3   Stv                     88 non-null     int64  
 4   Imenik                  88 non-null     int64  
 5   Potrdilo                88 non-null     int64  
 6   VOLIVCEV                88 non-null     int64  
 7   GImenik                 88 non-null     int64  
 8   GPotrdilo               88 non-null     int64  
 9   GLASOVALO               88 non-null     int64  
 10  Oddanih                 88 non-null     int64  
 11  Neveljavnih             88 non-null     int64  
 12  VELJAVNIH               88 non-null     int64  
 13  ZELENI                  88 non-null     int64  
 14  DeSUS                   88 non-null     int6

In [11]:
map = folium.Map(location = [46.119944, 14.815333], tiles='OpenStreetMap' , zoom_start = 9)
import folium.features
import folium.map

geo = df_vo.merge(dz18_vo, how='inner', on='VDV_ID')

geo

,ENOTA,VDV_ID,VDV_UIME,POV_KM2,STATUS,geometry,centroid,VE,VO,Stv,...,GSN_DELEZ,NPS_DELEZ,SPS_DELEZ,PIRATI_DELEZ,SDS_DELEZ,ZDRUŽENA LEVICA_DELEZ,SD_DELEZ,ZD_DELEZ,SMC_DELEZ,GAS_DELEZ
0,VO,1001,Volilni okraj Jesenice,374.88,V,"POLYGON ((13.90794 46.51983, 13.90772 46.51973...",POINT (14.06017 46.43657),76,76,3290683,...,0.578280,0.382686,0.552768,2.712816,23.292797,0.697338,8.878306,0.756867,8.606174,0.263628
1,VO,1002,Volilni okraj Radovljica 1,509.77,V,"POLYGON ((13.81248 46.23836, 13.81254 46.23836...",POINT (13.98173 46.42060),67,134,3290325,...,0.286981,0.169580,0.300026,2.674146,24.002087,0.300026,7.761544,0.704409,7.944169,0.260892
2,VO,1003,Volilni okraj Radovljica 2,131.20,V,"POLYGON ((14.26226 46.32228, 14.26258 46.32242...",POINT (14.17207 46.34252),65,195,3290270,...,0.622622,0.230601,0.288251,3.528191,23.590453,0.553442,8.290096,1.026173,8.647527,0.219071
3,VO,1004,Volilni okraj Kranj 1,29.64,V,"POLYGON ((14.35005 46.29351, 14.35027 46.29318...",POINT (14.35575 46.24364),54,216,3290040,...,0.341517,0.249215,0.221525,2.870593,22.161713,0.683035,7.458003,0.692265,8.925605,0.378438
4,VO,1005,Volilni okraj Kranj 2,86.72,V,"POLYGON ((14.39601 46.20523, 14.39546 46.20580...",POINT (14.24194 46.26868),60,300,3290150,...,0.384334,0.183016,0.228770,2.900805,22.053441,0.613104,7.046120,0.649707,9.095900,0.173865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,VO,8007,Volilni okraj Lenart,204.54,V,"POLYGON ((15.96478 46.56673, 15.96478 46.56674...",POINT (15.83238 46.57439),544,476,3290354,...,0.168722,0.000000,0.129786,1.271901,35.003245,0.765737,6.995457,0.895522,8.552888,0.168722
84,VO,8008,Volilni okraj Pesnica,189.86,V,"POLYGON ((15.75267 46.69789, 15.75181 46.69796...",POINT (15.56459 46.64180),488,488,3290199,...,0.419753,0.000000,0.160494,1.567901,35.320988,0.197531,8.234568,0.530864,8.098765,1.432099
85,VO,8009,Volilni okraj Ptuj 1,240.50,V,"POLYGON ((15.82751 46.45265, 15.82807 46.45221...",POINT (15.95530 46.45746),904,1017,3293094,...,0.324178,0.000000,0.279154,1.260693,30.670869,0.225124,6.429536,0.450248,5.691130,0.171094
86,VO,8010,Volilni okraj Ptuj 2,46.35,V,"POLYGON ((15.82960 46.45105, 15.82910 46.45140...",POINT (15.87086 46.41981),560,700,3290415,...,0.484635,0.000000,0.253332,1.883467,26.148254,0.220289,11.587179,0.517678,7.401696,0.231303


In [13]:


for candidate in ['SDS', 'SD', 'NSi', 'PIRATI']:
    # fg = folium.map.FeatureGroup(name=candidate)

    choropleth = folium.features.Choropleth(
        geo_data=geo[['VDV_UIME', 'VDV_ID', '{}_DELEZ'.format(candidate), 'geometry']],
        data=dz18_vo,
        columns=['VDV_ID', '{}_DELEZ'.format(candidate)],
        key_on='feature.properties.VDV_ID',
        fill_color='YlGnBu', 
        fill_opacity=0.7, 
        line_opacity=0.8,
        legend_name='Podpora {}'.format(candidate),
        name='Podpora {}'.format(candidate),
        nan_fill_color='white',
        smooth_factor=0,
        highlight=True, show=False).add_to(map)

    choropleth.geojson.add_child(
        folium.features.GeoJsonTooltip(['VDV_UIME', '{}_DELEZ'.format(candidate)], labels=True)
    )

    # fg.add_to(map)



folium.map.LayerControl(position='bottomright').add_to(map)

map.save("map_vo.html")